# Create the input features and output lables for the Machine Learning classification training

## CLEAN THIS ... EVENTUALLY

In [1]:
import sqlite3
import pandas as pd

In [2]:
cnx = sqlite3.connect('Database/Civ6CitySettledData.db')
cur = cnx.cursor()
print(cnx)
print(cur)

## Labels:

Use the specifically created database view to retrieve the per turn data collected. This view also aligns timelines, that is, takes care of cities settled in turn 2, or even turn 3. I was surprised to learn the AI actually moves settlers before settling.

In [3]:
sqlSelect = 'SELECT * FROM cityPerTurnView WHERE turns >= 1 and turns <= 50'
cityPt = pd.read_sql_query(sqlSelect, cnx)
print(cityPt.shape)

(13150, 29)


I have excluded Faith as it was simply too variable to use. Also, it doesn't appear to be a core yield in general, although it is situationally very useful.

In [4]:
yields = ['food', 'production', 'gold', 'science', 'culture']
#yields = ['food']
cityIds = list(cityPt['cityId'].unique())
labelsDf = pd.DataFrame(columns={'foodTotal', 'foodScore', 'productionTotal', 'productionScore',
                                 'goldTotal', 'goldScore', 'scienceTotal', 'scienceScore',
                                 'cultureTotal', 'cultureScore', 'cityTotal', 'cityScore'},
                        index=cityIds)
labelsDf.reset_index(level=0,inplace=True)
labelsDf.rename(columns={'index':'cityId'}, inplace=True)
labelsDf.sort_values(by='cityId', inplace=True)
labelsDf.index = pd.RangeIndex(len(labelsDf.index))
labelsDf.fillna(0.0, inplace=True)

# For analysis we want to have a binary output, that is, cityScore 0 or 1.
# To get there each "feature" needs to be quantiled ... so 2 quantiles are needed!!!

quantileInternal = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1]
quantileExternal = [0, .75, 1]

for yld in yields:
    columnName = "{}PerTurn".format(yld)
    cumulativeDf = cityPt[['cityId', 'turns', columnName]].pivot(index='turns',
                                                                 columns='cityId',
                                                                 values=columnName).cumsum()
    t50 = cumulativeDf.loc[50].to_frame()
    t50.reset_index(level=0, inplace=True)
    t50.rename(columns={50:"{}Total".format(yld)}, inplace=True)
    # join here via boolean map as using cityID, not index - see df.update below
    labelsDf.loc[labelsDf.cityId.isin(t50.cityId), "{}Total".format(yld)] = t50["{}Total".format(yld)]
    
    quantilesDf = pd.qcut(labelsDf["{}Total".format(yld)], quantileInternal, labels=False).to_frame()
    quantilesDf.rename(columns={"{}Total".format(yld):"{}Score".format(yld)}, inplace=True)
    # update works cause we have index alignment
    labelsDf.update(quantilesDf)
    labelsDf['cityTotal'] = labelsDf['cityTotal'] + labelsDf["{}Score".format(yld)]

quantilesDf = pd.qcut(labelsDf['cityTotal'], quantileExternal, labels=False).to_frame()
quantilesDf.rename(columns={'cityTotal':'cityScore'}, inplace=True)
# update works cause we have index alignment
labelsDf.update(quantilesDf)

for yld in yields:
    labelsDf['{}Score'.format(yld)] = pd.qcut(labelsDf['{}Total'.format(yld)], quantileExternal, labels=False)

print(labelsDf.head())

   cityId  cultureScore  cultureTotal  scienceTotal  scienceScore  foodTotal  \
0       1             0        101.97        182.68             0      268.0   
1       2             0        164.75        196.94             0      321.0   
2       3             0        153.38        180.63             0      469.0   
3       4             0        140.42        150.22             0      195.0   
4       5             0        141.82        185.06             0      500.0   

   cityScore  foodScore  cityTotal  goldTotal  productionScore  \
0        0.0          0       16.0      289.8                1   
1        1.0          0       32.0      660.7                1   
2        0.0          0       17.0      251.5                0   
3        0.0          0        6.0      262.5                0   
4        0.0          0       15.0      250.0                0   

   productionTotal  goldScore  
0           492.45          0  
1           490.85          1  
2           159.15        

In [5]:
labelsDf.to_csv('ModelInput/labels.csv', encoding='utf-8', index=False)

## Features:

Use the cityPlotsSettled data collected to prepare the features we intend using as input into the model. The category key idea is explained in the introduction.

We also need to add the percentage of tiles that has a river, as well as the cityHasRiver input.

In [6]:
sqlSelect = 'SELECT * FROM cityPlotsSettled'
#sqlSelect = 'SELECT * FROM cityPlotsSettled WHERE ownerCityId <> "None"'
# Start with city centre and ring 1
#sqlSelect = 'SELECT * FROM cityPlotsSettled WHERE ring == 0'
cityPs = pd.read_sql_query(sqlSelect, cnx)
print(cityPs.shape)
#print(cityPs.dtypes)

(4997, 15)


In [7]:
# Terrain and Feature categories - these never occur in isolation so group them!
cityPs['catTf'] = cityPs['terrain'] + cityPs['feature']
# Remove None, whitespace, and brackets
cityPs['catTf'] = cityPs['catTf'].apply(lambda s: s.replace('None', '').replace(' and ', '').replace(' ', ''))
cityPs['catTf'] = cityPs['catTf'].apply(lambda s: s.replace('(','').replace(')', ''))

## Features
#cityPs['catF'] = cityPs['feature']
## Remove None, whitespace, and brackets
#cityPs['catF'] = cityPs['catF'].apply(lambda s: s.replace(' ', '').replace('(','').replace(')', ''))

# And, a category key for resources...
# Consolidate luxury resources
cityPs['catR'] = cityPs['resource']
# Remove all "hidden" strategic resources
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Uranium', 'None').replace('Oil', 'None').replace('Niter', 'None'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Aluminum', 'None').replace('Coal', 'None').replace('Iron', 'None'))
# Consolidate Luxuries
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Dyes', 'Lux').replace('Silver', 'Lux').replace('Diamonds', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Tea', 'Lux').replace('Salt', 'Lux').replace('Olives', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Ivory', 'Lux').replace('Sugar', 'Lux').replace('Coffee', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Cotton', 'Lux').replace('Furs', 'Lux').replace('Whales', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Marble', 'Lux').replace('Jade', 'Lux').replace('Turtles', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Gypsum', 'Lux').replace('Mercury', 'Lux').replace('Tobacco', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Wine', 'Lux').replace('Truffles', 'Lux').replace('Incense', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Silk', 'Lux').replace('Citrus', 'Lux').replace('Spices', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Cocoa', 'Lux').replace('Pearls', 'Lux').replace('Amber', 'Lux'))
# Consolidate bonus resources
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Sheep', 'OtherBonus').replace('Bananas', 'OtherBonus'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Fish', 'OtherBonus').replace('Deer', 'OtherBonus'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Crabs', 'OtherBonus').replace('Copper', 'OtherBonus'))
# Remove None, whitespace, and brackets
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace(' ', '').replace('(','').replace(')', ''))

# Still have 30 different "categories" here...
#print(cityPs[['plotId','catT']].groupby('catT').count().sort_values(by='catT', ascending=True).count())
#print(cityPs[['plotId','catF']].groupby('catF').count().sort_values(by='catF', ascending=True).count())
#print(cityPs[['plotId','catR']].groupby('catR').count().sort_values(by='catR', ascending=True).count())

#print(list(cityPs['catT'].unique()))
#print(list(cityPs['catF'].unique()))
#print(list(cityPs['catR'].unique()))

#cols = ['cityId'] + list(cityPs['catT'].unique()) + list(cityPs['catF'].unique()) + list(cityPs['catR'].unique())
cols = ['cityId'] + list(cityPs['catTf'].unique()) + list(cityPs['catR'].unique())
#print(cols, "\n", type(cols))
#list(filter(None, list(cityPs['catR'].unique())))
featuresDf = pd.DataFrame(columns = cols)
featuresDf = featuresDf.astype('float')
print(featuresDf.shape)
print(featuresDf.head)

(0, 36)
<bound method NDFrame.head of Empty DataFrame
Columns: [cityId, PlainsHills, PlainsMountain, PlainsRainforest, Plains, PlainsHillsWoods, DesertHills, PlainsWoods, PlainsHillsRainforest, Grassland, CoastLake, GrasslandHills, DesertMountain, Desert, GrasslandWoods, GrasslandMarsh, Ocean, Tundra, Snow, GrasslandHillsWoods, GrasslandMountain, CoastLakeReef, DesertFloodplains, TundraWoods, TundraMountain, TundraHills, DesertOasis, TundraHillsWoods, None, OtherBonus, Lux, Wheat, Stone, Cattle, Horses, Rice]
Index: []

[0 rows x 36 columns]>


In [8]:
# category percentages
for cityId in cityPs['recordedCityId'].unique():
    plotsDf = cityPs[cityPs['recordedCityId'] == cityId]

    catTfSeries = round(plotsDf.groupby(['catTf'])['plotId'].count() / 19, 4)
#    catFSeries = plotsDf.groupby(['catF'])['plotId'].count()
    catRSeries = round(plotsDf.groupby(['catR'])['plotId'].count() / 19, 4)
                                
    featuresDf.loc[cityId, 'cityId'] = cityId
    for cat, val in catTfSeries.iteritems():
        featuresDf.loc[cityId, cat] = val
#    for cat, val in catFSeries.iteritems():
#        featuresDf.loc[cityId, cat] = val
    for cat, val in catRSeries.iteritems():
        featuresDf.loc[cityId, cat] = val
featuresDf.fillna(0.0, inplace=True)
print(featuresDf.head)

<bound method NDFrame.head of      cityId  PlainsHills  PlainsMountain  PlainsRainforest  Plains  \
1       1.0       0.1579          0.0526            0.0526  0.3158   
2       2.0       0.0526          0.0000            0.1579  0.1053   
3       3.0       0.0526          0.0000            0.0526  0.3158   
4       4.0       0.0526          0.0000            0.1053  0.2105   
5       5.0       0.0000          0.0000            0.0526  0.0000   
6       6.0       0.0000          0.0000            0.0000  0.0000   
7       7.0       0.2105          0.1053            0.2105  0.2632   
8       8.0       0.0000          0.0000            0.0526  0.1053   
9       9.0       0.0526          0.0526            0.2105  0.2105   
10     10.0       0.2632          0.1579            0.0000  0.1053   
11     11.0       0.1579          0.0526            0.3158  0.2105   
12     12.0       0.1053          0.0000            0.1053  0.3158   
13     13.0       0.0000          0.0000            0.0000  

In [9]:
# hasRiver percentage and cityHasRiver calculation...
for cityId in cityPs['recordedCityId'].unique():
    plotsDf = cityPs[cityPs['recordedCityId'] == cityId]
    
    featuresDf.loc[cityId, 'cityHasRiver'] = plotsDf[plotsDf['isCity'] == True].iloc[0,:].hasRiver

# Drop the "none" now
del featuresDf['None']

# Makes sure all values are ints as we only have ints!
#featuresDf['cityId'] = featuresDf['cityId'].astype('int')
#featuresDf['cityHasRiver'] = featuresDf['cityHasRiver'].astype('int')
featuresDf = featuresDf.astype('float')
featuresDf['cityId'] = featuresDf['cityId'].astype('int')
featuresDf['cityHasRiver'] = featuresDf['cityHasRiver'].astype('int')

In [10]:
print(featuresDf.head())
print(featuresDf.dtypes)

   cityId  PlainsHills  PlainsMountain  PlainsRainforest  Plains  \
1       1       0.1579          0.0526            0.0526  0.3158   
2       2       0.0526          0.0000            0.1579  0.1053   
3       3       0.0526          0.0000            0.0526  0.3158   
4       4       0.0526          0.0000            0.1053  0.2105   
5       5       0.0000          0.0000            0.0526  0.0000   

   PlainsHillsWoods  DesertHills  PlainsWoods  PlainsHillsRainforest  \
1            0.1053       0.0526       0.1579                 0.0526   
2            0.0526       0.1053       0.1053                 0.0000   
3            0.0000       0.0000       0.1053                 0.0000   
4            0.0526       0.0000       0.0526                 0.0000   
5            0.0000       0.0000       0.0000                 0.0000   

   Grassland      ...       DesertOasis  TundraHillsWoods  OtherBonus     Lux  \
1     0.0526      ...               0.0               0.0      0.0526  0.1053

In [11]:
featuresDf.to_csv('ModelInput/features.csv', encoding='utf-8', index=False)